In [1]:
import ee
import geemap
import geemap.colormaps
import ipywidgets as w
from IPython.display import display

In [2]:
import geemap.toolbar

cmp = geemap.colormaps.palettes

In [70]:
Map = geemap.Map(center=(-5,-64), zoom=5, #basemap = 'TERRAIN',
                 draw_ctrl=False, attribution_ctrl=False, data_ctrl=False)
Map.layout.height = 'auto'

# Select only toolbar controls "inspector[0]" and "change_basemap[8]"
Map.toolbar.children = (Map.toolbar.children[0], Map.toolbar.children[8], Map.toolbar.children[-2])
Map.toolbar.layout.grid_template_rows='32px'

def show_map(map2show):
    try:
        Map.remove_colorbar()
    except:
        pass
    
    img = map2show[0]
    vmin = map2show[1]
    vmax = map2show[2]
    mtype = map2show[3]
    
    
    if vmin>0:
        img = img.updateMask(img.gte(vmin)).rename('value')
    else:
        img = img.updateMask(img.gt(vmin)).rename('value')
    img = img.updateMask(img.lte(vmax)).rename('value')
    
    if mtype == 'binary':
        parvis = {'min': vmin, 'max': vmin, 'palette': cmp.viridis_r}
        img = img.where(img, 1)
        Map.addLayer(img, parvis, 'Inundation map')
    else:
        parvis = {'min': vmin, 'max': vmax, 'palette': cmp.viridis_r}
        Map.addLayer(img, parvis, 'Inundation map')
        Map.add_colorbar(parvis, label = None, discrete=False, orientation='vertical', layer_name = 'Inundation map')
    
    

# Description
title = w.HTML(value='<{size}>Amazon Inundation</{size}>'.format(size='h1'))
subtitle = w.HTML(value='<{size}>WebGIS platform | Version Beta</{size}>'.format(size='h4'))
description = w.HTML(value=
'<{size}>The Amazon River basin is the largest on Earth, and harbors unique freshwater ecosystems on which thousands of people rely for their livelihoods, '
                     'contrasting ecosystems are adapted to its seasonal inundation dynamics, and major biogeochemical processes occur. <br/>'
                     
'The Amazon-Inundation WebGIS App provides an easy way to visualize the inundation extent across the basin based on various datasets that were developed in the last decades. <br/>'
                     
'The products have different data sources, from passive microwave and synthetic aperture radar to optical imagery and hydrological modeling, with spatial resolution varying from 12.5 m to 25 km. <br/>'

'Some products are dynamic, which are shown here as long-term flood frequency maps, i.e., the higher the pixel value, the longer it is flooded throughout the hydrological cycle. <br/>'

'Two 1 km consensus maps are also provided, which show, for each 1 km pixel, how many datasets agree that it is flooded considering the long-term maximum or minimum inundation maps. <br/>'

'References and data availability are provided for each dataset. For more details, and to cite this tool, please refer to Fleischmann et al. (2021). <br/></{size}>'
                     .format(size='p'))

#logos = w.HBox([logo_ird])
logos = w.HTML(value='<{size}><br/><br/><br/>Here will be the logos of the organizations that contributed with this development.<br/><br/><br/><br/></{size}>'.format(size='p'))

# Categories
catList = ['Agreement among datasets at 1 km', 'Regional/local scale', 'Basin scale']
catW = w.widgets.RadioButtons(options=catList, description='Category: ')

# Maps
mapList = {'Agreement among datasets at 1 km':
            {'Maximum extent agreement': [ee.Image('users/rbfontana/inundation_amazon/AgreementMax'), 1, 14, 'frequency',
                 'Products agreement on maximum inundation (number of products).'
             ],
             'Minimum extent agreement': [ee.Image('users/rbfontana/inundation_amazon/AgreementMin'), 1, 10, 'frequency',
                 'Products agreement on minimum inundation (number of products).'
             ]
             },
           
           'Regional/local scale':
            {'Uatumã (Resende et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/Resende'), 0, 999, 'binary',
                 'Binary.'
             ],
             'Curuai (Arnesen et al., 2103)': [ee.Image('users/rbfontana/inundation_amazon/Arnesen_FloodFrequency'), 0, 1, 'frequency',
                 'Flood frequency, in rate.'
             ],
             'Mamirauá (Ferreira-Ferreira et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/Mamiraua_FloodDur'), 1, 7, 'frequency',
                 'Flood frequency, in days per year, where: <br/>'
                 '&emsp; 1 &emsp;    <40 days, <br/>'
                 '&emsp; 2 &emsp;    40-105 days, <br/>'
                 '&emsp; 3 &emsp;    105-125 days, <br/>'
                 '&emsp; 4 &emsp;    125-175 days, <br/>'
                 '&emsp; 5 &emsp;    175-295 days, <br/>'
                 '&emsp; 6 &emsp;    295-365 days, <br/>'
                 '&emsp; 7 &emsp;    365 days. <br/>'
             ],
             'Janauacá (Pinel et al., 2019)': [ee.Image('users/rbfontana/inundation_amazon/PinelFloodFrequency'), 0, 1, 'frequency',
                 'Flood frequency, in rate.'
             ],
             'Llanos de Moxos (Ovando et al. 2016)': [ee.Image('users/rbfontana/inundation_amazon/ovandoMODIS'), 0, 1, 'frequency',
                 'Flood frequency, in rate.'
            ],
            },
           
           'Basin scale':
            {'ESA-CCI (Bontemps et al., 2013)': [ee.Image('users/rbfontana/inundation_amazon/esa20101500'), 150, 999, 'binary',
                 'Binary.'
             ],
             'GLWD (Lehner & Döll, 2004)': [ee.Image('users/rbfontana/inundation_amazon/glwd_Amaz500'), 0, 999, 'binary',
                 'Binary.'
             ],
             'CIFOR (Gumbricht et al., 2017) ': [ee.Image('users/rbfontana/inundation_amazon/cifor_amaz500'), 0, 999, 'binary',
                 'Binary.'
             ],
             'MGB (Siqueira et al., 2018)': [ee.Image('users/rbfontana/inundation_amazon/MGB_AmazonBasin'), 0, 1, 'frequency',
                 'Flood frequency, in rate.'
             ],
             'G3WBM (Yamazaki et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/g3wbm_amaz500'), 0, 999, 'binary',
                 'Binary.'
             ],
             'GIEMS-D15 (Fluet-Chouinard et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/giems_amaz21500'), 0, 999, 'binary',
                 'Binary.'
             ],
            'Chapman (Chapman et al., 2015)': [ee.Image('users/rbfontana/inundation_amazon/chapman_amaz_compres500'), 0, 4, 'binary',
                 'Binary.'
            ],
            }
        }

mapDwn = w.widgets.Dropdown(options=mapList[catW.value], description='Map: ',
                    )
def update_mapDwn(cat):
    mapDwn.options = mapList[cat.new]
catW.observe(update_mapDwn, 'value')
    
input_ops = w.interactive(show_map, map2show=mapDwn)

map_description = w.HTML(value = str(mapDwn.value[-1]))
def update_map_description(product):
    map_description.value = str(product.new[-1])
mapDwn.observe(update_map_description, 'value')

carousel_layout = w.Layout(#overflow='scroll hidden',
                    #border='3px solid black',
                    width='600px',
                    height='100%',
                    flex_flow='column',
                    display='flex')

carousel = w.Box(children=[title, subtitle, description, logos, catW, input_ops, map_description], layout=carousel_layout)
carousel.layout.display = 'block' #Blocks the display visualization so items wont be hidden

box_layout = w.Layout(
    height='94vh',  # viewport height
    width='100%',
    grid_gap='5px',
    overflow='auto',
    align_items='stretch',
    margin='0px',
    border='0px',
    padding='0px',
)
box = w.HBox([carousel, Map], layout=box_layout)

display(box)
#Map